# Xử lý Null

### Product

In [ ]:
import pandas as pd
product = pd.read_csv(r'D:\Document\doan2\data-doan2\product_list.csv')

In [ ]:
columns_to_drop = [
    'gift_item_title',
    'current_seller_name',
    'current_seller_link',
    'current_seller_store_id'
]

product = product.drop(columns=columns_to_drop)


In [ ]:
null_values = product.isnull().sum()
null_values

In [ ]:
product = product.dropna(subset=['current_seller_id'])

### Store

In [ ]:
store = pd.read_csv(r'D:\Document\doan2\data-doan2\store_list.csv')

In [ ]:
store.shape
null_values = store.isnull().sum()
null_values

In [ ]:
updated_data = []

for index, row in null_rating.iterrows():
    seller_url = row['seller_link']
    
    driver.get(seller_url)

    try:
        badge_wrapper = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'Header__BadgeWrapper-sc-vgcg69-11.czboZb'))
        )

        sub_info = badge_wrapper.find_element(By.CLASS_NAME, 'Header__SubInfo-sc-vgcg69-12.iUmIsA')
        
        rating_text = sub_info.text.split('/')[0].strip()  
        updated_data.append((row['seller_id'], rating_text))
        
    except Exception as e:
        continue

for id, rating in updated_data:
    store.loc[store['seller_id'] == id, 'rating_seller'] = rating

store = store.dropna(subset=['rating_seller'])

### Customer

In [ ]:
customer = pd.read_csv(r'D:\Document\doan2\data-doan2\customer_list.csv')

In [ ]:
columns_to_keep = [
    'created_by_id', 
    'created_by_name',
    'contribute_info_joined_time', 
    'contribute_info_total_review', 
    'contribute_info_total_thank'
]

customer = customer[columns_to_keep]

null_values = customer.isnull().sum()
null_values
customer = customer.dropna(subset=['created_by_id'])

### Review

In [ ]:
review = pd.read_csv(r'D:\Document\doan2\data-doan2\review_list.csv')
null_values = review.isnull().sum()
null_values
columns_to_keep = [
    'id',
    'title',
    'created_by_id',
    'content',
    'status',
    'rating',
    'created_at',
    'product_id',
    'timeline_review_created_date'
]
review = review[columns_to_keep]
review = review.dropna(subset=['created_by_id'])

### Orders

In [ ]:
order = pd.read_csv(r'D:\Document\doan2\data-doan2\order_list.csv')
null_values = order.isnull().sum()
null_values
order = order.dropna(subset=['created_by_id', 'timeline_delivery_date'])

# Xử lý trùng lặp

In [ ]:
import pandas as pd
product = pd.read_csv(r'D:\Document\doan2\data-doan2-clean\product_clean.csv')
product = product.drop_duplicates(subset='id', keep='first')
product = product.drop_duplicates()
store = store.drop_duplicates()
customer = customer.drop_duplicates()
review = review.drop_duplicates()
order = order.drop_duplicates()


# Chuyển đổi kiểu dữ liệu

In [ ]:
product.info()
product['all_time_quantity_sold'] = product['all_time_quantity_sold'].fillna(0).astype(int)
product['brand_id'] = product['brand_id'].fillna(0).astype(int)
product['current_seller_id'] = product['current_seller_id'].fillna(0).astype(int)

In [ ]:
breadcrumbs_list = product['breadcrumbs'].tolist()

categories_list = [cat.split(', ') for cat in breadcrumbs_list]

def build_hierarchy(categories):
    hierarchy = {}
    id_map = {}
    parent_map = {}
    next_id = 1

    for category in categories:
        current_level = hierarchy
        parent_id = None
        for cat in category:
            if cat not in current_level:
                current_level[cat] = {'subcategories': {}}
                id_map[cat] = next_id 
                parent_map[next_id] = parent_id
                next_id += 1
            current_level = current_level[cat]['subcategories']
            parent_id = id_map[cat]
            parent_map[next_id] = parent_id

    return id_map, parent_map

id_map, parent_map = build_hierarchy(categories_list)

categories_df = pd.DataFrame(id_map.items(), columns=['category_name', 'category_id'])
categories_df['parent_id'] = categories_df['category_id'].map(parent_map)

product['breadcrumbs_id'] = product['breadcrumbs'].apply(
    lambda x: [id_map[cat] for cat in x.split(', ') if cat in id_map]
)

product['breadcrumbs_id'] = product['breadcrumbs_id'].apply(lambda x: ', '.join(map(str, x)))

product.drop(columns=['breadcrumbs'], inplace=True)

categories_df['parent_id'] = categories_df['parent_id'].fillna(0).astype(int)

In [ ]:
store.info()
store['rating_seller'] = pd.to_numeric(store['rating_seller'], errors='coerce')

In [ ]:
review.info()
review['created_by_id'] = review['created_by_id'].fillna(0).astype(int)
review['timeline_review_created_date'] = pd.to_datetime(review['timeline_review_created_date'], errors='coerce')
review.iloc[100]
review['timeline_review_created_date'] = pd.to_datetime(review['timeline_review_created_date'])

review['date'] = review['timeline_review_created_date'].dt.date 
review['time'] = review['timeline_review_created_date'].dt.time

review.drop(columns=['timeline_review_created_date'], inplace=True)

review['date'] = review['date'].astype(str) 

review['time'] = review['time'].astype('object') 
review['date'] = pd.to_datetime(review['date'])

In [ ]:
customer.info()
customer['created_by_id'] = customer['created_by_id'].astype('Int64') 

customer['created_by_name'].fillna('Unknown', inplace=True)

customer['contribute_info_total_review'] = customer['contribute_info_total_review'].astype(int)
customer['contribute_info_total_thank'] = customer['contribute_info_total_thank'].astype(int)

In [ ]:
order.info()
order['created_by_id'] = order['created_by_id'].astype('Int64') 
order['created_by_name'].fillna('Unknown', inplace=True)
order['timeline_delivery_date'] = pd.to_datetime(order['timeline_delivery_date'], errors='coerce')

# Lưu trữ dữ liệu

In [ ]:
categories_df.to_csv('categories_clean.csv', index=False)
product.to_csv('product_clean.csv', index=False)
store.to_csv('store_clean.csv', index=False)
customer.to_csv('customer_clean.csv', index=False)
review.to_csv('review_clean.csv', index=False)
order.to_csv('order_clean.csv', index=False)